# 获取2014世界杯的详细比赛数据

[@寒小阳](http://blog.csdn.net/han_xiaoyang)<br>
2015-05

关于世界杯比赛的粗粒度数据，好像还是挺多地方能找到的，如果自己想做细致的分析，比如对于每场比赛的断球、过人、传球和射门等等都记录下来，好像比较少有这么详细的数据报道。<br>
不过世界杯还是世界杯，你还是可以看到一些网站，比如[Huff Post Data's World Cup page](http://data.huffingtonpost.com/2014/world-cup)。<br>
大家之前都学过爬虫对吧，咱们动动手，把数据组织下来吧。

In [1]:
#imports
import requests
import json
import mechanize
from bs4 import BeautifulSoup
import time

#initializes the browser
br = mechanize.Browser()
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=10)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]


当然是想要获取所有比赛的子链接咯，你仔细看HTML源码的话，会发现在一个"matchup"的tag里，下面这段代码应该能帮助你们想起爬取解析的过程。

In [2]:
starting_link = 'http://data.huffingtonpost.com/2014/world-cup/matches/belgium-vs-usa-731822'
response = mechanize.urlopen(starting_link)
soup = BeautifulSoup(response)

links_html = soup.find_all("span", class_="matchup")

links = []

for link_html in links_html:
    a = link_html.find_all('a')
    for l in a:
        link = l.get('href')
        link = link.split('/')[-1]

    links.append(link)
    
links[:5]

/Library/Python/2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


['brazil-vs-chile-731815',
 'colombia-vs-uruguay-731816',
 'netherlands-vs-mexico-731817',
 'costa-rica-vs-greece-731818',
 'france-vs-nigeria-731819']

我们用一个json模块把它转成可读的清晰结构

In [3]:
def get_match_data(match):
    match_id = match.split('-')[-1]
    response = mechanize.urlopen('http://data.huffingtonpost.com/2014/world-cup/matches/%s.json' % match_id)

    match_data = json.loads(response.read())
    
    return match_data

match_data = get_match_data(links[0]) # test

match_data.keys()

[u'team_stats', u'events', u'summary']

In [4]:
match_data

{u'events': [{u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'48852',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'122779',
   u'to_x': None,
   u'to_y': None,
   u'to_z': None,
   u'type': u'102',
   u'x': 0.0,
   u'y': 0.0},
  {u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'122779',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'37269',
   u'to_x': None,
   u'to_y': None,
   u'to_z': None,
   u'type': u'102',
   u'x': 0.0,
   u'y': 0.0},
  {u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'37269',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'48853',
   u

尴尬的是，你会发现拿到的数据里只有ID，而没有球员的名字，所以咱们得找找，怎么把ID和球员的名字对应起来。

我们在主页面的HTML源码里会发现 HPIN.teams 和 HPIN.players 包含了名字和ID，当然了，还有一堆附加的信息（比如位置，生日，习惯用左脚还是右脚等等），咱们来解析一下

In [5]:
def get_match_names(match):
    response = mechanize.urlopen('http://data.huffingtonpost.com/2014/world-cup/matches/%s' % links[0]) #example page
    soup = BeautifulSoup(response)

    data = {}

    data_script = soup.findAll("script")[1] #gets the second script block. Hopefully all pages follow the same format
    data_lines = data_script.text.split('\n')

    for line in data_lines[1:]:
        try:
            #format of a variable is HPIN.variable = [list of dictionaries]
            #this tries to convert it to 
            line_data = line.split(' = ')
            name = line_data[0].split('.')[1]
            value = json.loads(line_data[1][:-1])
            data[name] = value
        except:
            print "Pass this line...", line #should only occur on blank lines - yeah, I know, lazy exception handling...
            
    return data

names = get_match_names(links[0])
names.keys()

Pass this line... 


[u'statCategories',
 u'awayTeam',
 u'callbackPath',
 u'homeTeam',
 u'teams',
 u'players',
 u'imageCallbackPath',
 u'imageCallbackInterval',
 u'twitterUrl']

OK，所以我们现在有所有场次比赛的链接，也写了一个函数用于返回比赛和比赛里的一些信息，另外一个函数用于返回球员和球队名字。我们合并一下，来做个python的字典。

In [6]:
data = {}

Then, execute a loop that will get the data from all the matches and add it to the dictionary. The `if` statement ensures you don't have to reprocess a match in the case you have to run the cell again (e.g. due a network error).

In [ ]:
for match in links:
    if match not in data:
        print match
        time.sleep(10)

        match_data = get_match_data(match)
        match_names = get_match_names(match)
        data[match] = {'data': match_data, 'names': match_names}

        print match, " done" 
    else:
        print match, " already processed"

In [ ]:
print len(data.keys()) #make sure you have all 64 games

In [10]:
data['germany-vs-algeria-731820']

{'data': {u'events': [{u'displaymin': 1,
    u'event_id': None,
    u'field_pass': u'f',
    u'id': None,
    u'min': 0,
    u'outcome': u'1',
    u'period': u'1',
    u'player_id': u'69140',
    u'qualifiers': {},
    u'sec': 0,
    u'side': u'H',
    u'team': u'357',
    u'to': u'41316',
    u'to_x': None,
    u'to_y': None,
    u'to_z': None,
    u'type': u'102',
    u'x': 0.0,
    u'y': 0.0},
   {u'displaymin': 1,
    u'event_id': None,
    u'field_pass': u'f',
    u'id': None,
    u'min': 0,
    u'outcome': u'1',
    u'period': u'1',
    u'player_id': u'41316',
    u'qualifiers': {},
    u'sec': 0,
    u'side': u'H',
    u'team': u'357',
    u'to': u'15207',
    u'to_x': None,
    u'to_y': None,
    u'to_z': None,
    u'type': u'102',
    u'x': 0.0,
    u'y': 0.0},
   {u'displaymin': 1,
    u'event_id': None,
    u'field_pass': u'f',
    u'id': None,
    u'min': 0,
    u'outcome': u'1',
    u'period': u'1',
    u'player_id': u'15207',
    u'qualifiers': {},
    u'sec': 0,
    u'si

In [29]:
import pickle

pickle.dump(data, open( "wc2014.p", "wb"))

In [32]:
data == pickle.load(open("wc2014.p", "rb")) #because I'm a bit OCD and want to make sure the data was properly stored

True

OK，数据有了，可以准备开始玩耍了。